In [5]:
import numpy as np

In [6]:
#Including numba to make the program more effective
import numba as nb
from numba import jit
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

In [7]:
@jit(nopython=True)
def find_min_collision_time_wall(particles_involved, collision_count):
    '''
    Parameters:
        particles = [[posx1, posy1, velx1, vely1, mass1, radius1]
                     [posx2, posy2, velx2, vely2, mass2, radius2]
                     ...
                     [posxN, posyN, velxN, velyN, massN, radiusN]]

    Output:
        min_collision_times = [[collision time, particle, ,other particle, wall]]
        wall_colliding = [particle1, particle2, ...]

    Calculates the minimun time till collision for each particle. Returns all times in an array that at 
    a later point will be prioritized. To be used only one the first time? After collision maybe we only 
    need to calculate new collisions for the particles that collided and not all particles.
    '''
    
    min_collision_times = np.zeros((len(particles_involved),6))                   

    for i in range(len(particles_involved)):


        wall = 2
        other_particle = i
        times_till_walls = np.zeros(2) #Time till verical and horizontal wall
      
        pos_x = particles_involved[i, 0]
        pos_y = particles_involved[i, 1]
        vel_x = particles_involved[i, 2]
        vel_y = particles_involved[i, 3]
        radius = particles_involved[i, 5]

        #Vertical walls:
        if vel_x > 0:
            times_till_walls[0] = np.abs((1 - radius - pos_x)/vel_x)
        elif vel_x < 0:
            times_till_walls[0] = np.abs((radius - pos_x)/vel_x)
        else:
            times_till_walls[0] = np.inf


        #Horizontal walls:
        if vel_y > 0:
            times_till_walls[1] = np.abs((1 - radius - pos_y)/vel_y)
        elif vel_y < 0:
            times_till_walls[1] = np.abs((radius - pos_y)/vel_y)
        else:
            times_till_walls[1] = np.inf

        min_collision = np.amin(times_till_walls)
        if min_collision == times_till_walls[0]:
            wall = 1                                           #VERTICAL = 1
            
        elif min_collision == times_till_walls[1]:
            wall = 0                                           #HORIZONTAL = 0

        min_collision_times[i] = [min_collision, i, other_particle, wall, collision_count[i],collision_count[other_particle]]          
        
        # [time, which particle (0,1,2,3,...,N), 0,which wall, collision count1, collisioncount2,]
        
    return min_collision_times

In [10]:
#@jit(nopython=True)
def after_collision_wall(particles, particle1, wall, timeCollision, v0, collision_count, xi = 1):
    '''
    Parameters:
        particles = [[posx1, posy1, velx1, vely1, mass1, radius1],
                     [...]]
        particle1 = The index of the particle that collided
        wall =  0 (horizontal wall), 1 (vertical wall) or 2 (particle)
        xi =  Elasticity

    Output:
        particles = [[posx1, posy1, velx1, vely1, mass1, radius1],
                     [...]]
        new_collisiontime = Array of all calculates collision times
        particle1 = The index of the particle that collided

    This function finds the new velocity of the particle after the collision as well as new collision times!
    '''
    velx_old = float(particles[particle1][2])
    vely_old = float(particles[particle1][3])
    posx_old = float(particles[particle1][0])
    posy_old = float(particles[particle1][1])
    
    if wall == 0:     #horis
        velx_new = xi * velx_old
        vely_new = -xi * vely_old
    elif wall == 1:   #verti
        velx_new = -xi*velx_old
        vely_new = xi * vely_old
    
    if timeCollision == np.inf:                                          
        posx_new = posx_old
        posy_new = posy_old
    else:
        posx_new = posx_old + (velx_old * timeCollision)                 # they only travel in straight lines; so strekning = fart*tid 
        posy_new = posy_old + (vely_old * timeCollision)                 # ENDRA TIL NEW


    particles[particle1][2] = velx_new                                   # put the new velocities & positions into particles
    particles[particle1][3] = vely_new    
    particles[particle1][0] = posx_new                                   # put the new velocities & positions into particles
    particles[particle1][1] = posy_new 
    
    particles = updateParticles(particles, np.array([particle1]), timeCollision)

    new_collisiontime = find_min_collision_time_wall(particles, collision_count)  # find new collision times for wall. 2D to work
    
    return particles, new_collisiontime, particle1

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d17ccd91-5f3d-4d77-a625-6b1eb51e3637' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>